In [44]:
using JLD2

include("../src/filters.jl")
include("../src/reduction.jl")

@JLD2.load "variables/train_dataset.jld2" train_inputs train_targets
@JLD2.load "variables/test_dataset.jld2" test_inputs test_targets

2-element Vector{Symbol}:
 :test_inputs
 :test_targets

### Selección de Características

#### kendall-Tau

In [31]:
if isfile("variables/kendall_coeffiecients.jdl2")
    @JLD2.load "variables/kendall_coeffiecients.jdl2" kendall_coeff
else
    kendall_coeff = kendall((train_inputs, train_targets))
    @save "variables/kendall_coeffiecients.jdl2" kendall_coeff
end;
kendall_index = findall(x -> x >= 0.3, kendall_coeff);
new_dataset = (train_inputs[:, kendall_index], train_targets);

#### ANOVA

In [ ]:

anova_scores = anova((train_inputs, train_targets));
anova_index = findall(x -> x >= 0.3, anova_scores);
# idx = sortperm(scores, rev=true)[1:k]
new_dataset = (train_inputs[:, anova_index], train_targets);

(9245×412 DataFrame
  Row │ tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X ⋯
      │ Float64?          Float64           Float64?          Float64?         ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │       0.0023607         0.0084447         0.0432368         0.00244396 ⋯
    2 │       0.00306771        0.0112894         0.00533177        0.00224904
    3 │       0.00149202        0.0111599         0.00142985        0.0012169
    4 │       0.00153559        0.00903754        0.00164208        0.00120711
    5 │       0.00120025        0.00737174        0.0021737         0.0010343  ⋯
    6 │       0.00122669        0.00642213        0.00198672        0.00122207
    7 │       0.00199689        0.0085256         0.00175186        0.00180589
    8 │       0.00202387        0.0101122         0.00364083        0.00183641
  ⋮   │        ⋮                 ⋮                 ⋮                 ⋮         ⋱
 9239 │       0.30349

#### Pearson

In [ ]:
filtered_train_inputs, correlations = pearson((train_inputs, train_targets), threshold=0.1);
new_dataset = (filtered_train_inputs, train_targets);

(9245×498 DataFrame
  Row │ tBodyAcc-entropy()-Y  tBodyAccMag-sma()  tGravityAccMag-arCoeff()3  fB ⋯
      │ Float64               Float64?           Float64                    Fl ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │             0.160331         0.0202831                    0.547416     ⋯
    2 │             0.30932          0.00545572                   0.479121
    3 │             0.289588         0.00362462                   0.607213
    4 │             0.2346           0.00270491                   0.649787
    5 │             0.249538         0.00223164                   0.473595     ⋯
    6 │             0.209004         0.00172323                   0.577158
    7 │             0.317816         0.00377577                   0.811336
    8 │             0.287624         0.0038835                    0.684536
  ⋮   │          ⋮                    ⋮                      ⋮                 ⋱
 9239 │             0.632951         0.39390

#### Spearman

In [ ]:
spearman_scores = spearman((train_inputs, train_targets));
spearman_index = findall(x -> x >= 0.01, spearman_scores);
new_dataset = (train_inputs[:, spearman_index], train_targets);
print(size(new_dataset[1], 2))

442

#### Mutual information

In [ ]:
### SIN IMPLEMENTAR AÚN

### RFE (con regresión logística)

In [ ]:
### NO FUNCIONA!

filtered_features = rfe_logistic((train_inputs, train_targets));
new_dataset = (train_inputs[:, filtered_features], train_targets)

### Reducción de dimensionalidad

#### PCA

In [35]:
X_pca, pca_model, k = pca(train_inputs, 0.95)
new_dataset = (X_pca, train_targets)

(9245×66 DataFrame
  Row │ x1        x2          x3            x4           x5          x6        ⋯
      │ Float64   Float64     Float64       Float64      Float64     Float64   ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │  2.76153  -0.256987    0.734565      0.767155     0.696267    0.143982 ⋯
    2 │  2.71893   0.0640548   0.614245      0.0616313   -0.114413   -0.093028
    3 │  3.11647   0.608182    0.769944      0.490605     0.52655    -0.320175
    4 │  3.12132   0.565492    0.678904      0.554163     0.438771   -0.308241
    5 │  3.09861   0.708369    0.748107      0.389798     0.473601   -0.319154 ⋯
    6 │  3.08885   0.428344    0.414696      0.517041     0.402857   -0.389024
    7 │  2.95899   0.709203    0.634724      0.260148     0.126266   -0.254458
    8 │  2.98497   0.860147    0.65654       0.13619      0.0401643  -0.284319
  ⋮   │    ⋮          ⋮            ⋮             ⋮           ⋮           ⋮     ⋱
 9239 │ -2.47426  -0.

#### LDA

In [ ]:
X_lda, W = lda((train_inputs, train_targets))
new_dataset = (X_lda, train_targets)

1


#### ICA

In [ ]:
S, W = fastica((train_inputs, train_targets); tol=0.01, max_iter=100)
new_dataset = (S, train_targets)

561
